In [1]:
import io
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile

#### Q1. Socket

Bạn là một nhà phát triển, làm việc trên một chương trình Python 3. Bạn được yêu cầu thực hiện tích hợp liên tục bằng cách sử dụng sockets, để gửi dữ liệu giữa các ứng dụng. Hãy xem xét đoạn mã sau từ ứng dụng của bạn:

In [ ]:
# .. some code here
try:
    connection._extra['sockname'] = socket.getsockname()
except socket.error as e:
    connection._extra['sockname'] = None
    logger.error("Error getting socket name: %s", e)

if "peername" not in connection._extra:
    try:
        connection._extra['peername'] = socket.getpeername()
    except socket.error as e:
        connection._extra['peername'] = None
        logger.error("Error getting peer name: %s", e)
# .. some code here


Câu hỏi: Phát biểu nào sau đây đúng về đoạn mã trên?
- a. Một số UDP sockets có thể không có tên socket, và do đó chúng ta đang bắt ngoại lệ cho tất cả các UDP sockets.
- b. Một số UDP sockets có thể có hoặc không có tên peer, và do đó chúng ta đang bắt ngoại lệ bằng cách sử dụng socket.error.
- c. Một số TCP sockets có thể không có tên peer, và do đó chúng ta đang bắt ngoại lệ bằng cách sử dụng socket.error.
- d. Chúng ta có thể trực tiếp sử dụng socket.getpeername() mà không cần khối try except, vì getpeername trả về một tuple và xử lý null nếu không có.

#### Q2.

In [13]:
# data = content
# data = pd.read_csv(r'C:\Users\admin\Downloads\data\cardio_base.csv')

df = pd.read_csv('https://raw.githubusercontent.com/Hiienng/ForSharing/main/cardio_base.csv')

In [14]:
# 1. The gap between age group with highest weight and the group with lowest weight

df['age_year'] = df['age'] // 360

df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke,age_year
0,0,18393,2,168,62.0,110,80,1,0,51
1,1,20228,1,156,85.0,140,90,3,0,56
2,2,18857,1,165,64.0,130,70,3,0,52
3,3,17623,2,169,82.0,150,100,1,0,48
4,4,17474,1,156,56.0,100,60,1,0,48
...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,53
69996,99995,22601,1,158,126.0,140,90,2,0,62
69997,99996,19066,2,183,105.0,180,90,3,0,52
69998,99998,22431,1,163,72.0,135,80,1,0,62


.mean(axis=0): Tính trung bình theo cột (axis=0 là mặc định)
.mean(axis=1): Tính trung bình theo hàng
.mean(skipna=True): Bỏ qua các giá trị NaN khi tính trung bình (mặc định là True)
.mean(numeric_only=True): Chỉ tính trung bình các cột/hàng có kiểu dữ liệu số
.idxmax()

In [16]:
# Tính trọng lượng trung bình theo tuổi
df_2 = df.groupby('age_year')['weight'].mean().reset_index(name = 'average_weight')

# Tìm tuổi có trọng lượng trung bình cao nhất và thấp nhất
max_age = df_2.loc[df_2['average_weight'].idxmax()]['age_year']
min_age = df_2.loc[df_2['average_weight'].idxmin()]['age_year']

# Tính khoảng cách tuổi
age_gap = max_age - min_age

In [18]:
max_age

29.0

In [1]:
# 2. Do people over 50 higher cholesterol than the rest?

In [20]:
df['OVER_50'] = df['age_year']>=50
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke,age_year,OVER_50
0,0,18393,2,168,62.0,110,80,1,0,51,True
1,1,20228,1,156,85.0,140,90,3,0,56,True
2,2,18857,1,165,64.0,130,70,3,0,52,True
3,3,17623,2,169,82.0,150,100,1,0,48,False
4,4,17474,1,156,56.0,100,60,1,0,48,False
...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,53,True
69996,99995,22601,1,158,126.0,140,90,2,0,62,True
69997,99996,19066,2,183,105.0,180,90,3,0,52,True
69998,99998,22431,1,163,72.0,135,80,1,0,62,True


In [21]:
df_over_50 = df.groupby('OVER_50')['cholesterol'].mean().reset_index(name = 'w_cholesterol')
df_over_50

,OVER_50,w_cholesterol
0,False,1.229957
1,True,1.414926


In [28]:
# 3. Find the average systolic (ap_hi) and diastolic (ap_lo) blood pressure for each gender.

df_ap = df.groupby('gender')[['ap_hi', 'ap_lo']].mean().reset_index()
df_ap


,gender,ap_hi,ap_lo
0,1,128.139249,94.522776
1,2,130.078872,100.551982


In [29]:
# 4. Identify any significant differences in average blood pressure between men and women.
df_blood = df.groupby('gender')['ap_hi'].mean().reset_index()
df_blood

,gender,ap_hi
0,1,128.139249
1,2,130.078872


In [31]:
# Analyze the percentage of smokers in the dataset.
smoker_per  = df[['smoke'] == 1]['smoke'].sum() / len(df) 
smoker_per_2  = df[['smoke']].sum() / len(df) 
smoker_per
# Determine if there's a significant difference in average cholesterol levels between smokers and non-smokers.

0.08812857142857143

In [36]:
df['smoke'] == 1

0        False
1        False
2        False
3        False
4        False
         ...  
69995     True
69996    False
69997    False
69998    False
69999    False
Name: smoke, Length: 70000, dtype: bool

In [35]:
len(df[df['smoke'] == 1])

KeyError: False